In [3]:
from opensky_api import OpenSkyApi
from kafka import KafkaProducer
import json
from time import sleep
    
# # Topics/Brokers
real_time_states = 'real-time-states'
raw_json = 'raw_json'
flat_json = 'flat_json'
brokers = ['localhost:9092']

def open_sky_generator(api):
    while True:
        yield api.get_states()
        sleep(10)


In [4]:
producer = KafkaProducer(bootstrap_servers=brokers)

In [ ]:
def opensky_to_json(opensky_states):
    """returns a list of jsons"""
    for state in opensky_states:
        opensky_json = 

In [ ]:
producer.send()